In [2]:
import pymongo

In [3]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')

In [4]:
db = client.EUDemographics

In [5]:
db.countries.update({}, {'$unset': {'intermediate-region-code':1}} , multi=True)
db.countries.update({}, {'$unset': {'iso_3166-2':1}} , multi=True)
db.countries.update({}, {'$unset': {'intermediate-region':1}} , multi=True)
db.countries.update({}, {'$unset': {'sub-region-code':1}} , multi=True)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  """Entry point for launching an IPython kernel.
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  after removing the cwd from sys.path.


{'n': 248, 'nModified': 248, 'ok': 1.0, 'updatedExisting': True}

In [33]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

print(similar("United states", "united kingdom"))

0.5185185185185185


In [173]:
def find_country_code(name):
    db = client.EUDemographics
    countries = db.countries.find({})
    distances = []
    for post in countries:
        predistances = []
        predistances.append(similar(post['name'],name))
        predistances.append(similar(post['alpha-2'],name))
        predistances.append(similar(post['alpha-3'],name))
        alternatives = post.get('alternative_names')
        if alternatives:
            for nom in post.get('alternative_names'):
                predistances.append(similar(nom,name))
        distances.append([max(predistances),post['alpha-2'],post['name']])
    f = sorted(distances,key=lambda x: (x[0],x[1]))[-1]
    if f[0] > 0.95:
        return f[1], f[2]
    else :
        response = input(str(f[1]) +  " " + str(f[2]) + " =? " + name + '\n')
        if response == 'y':
            db.countries.update_one({"alpha-2" : f[1]}, {"$addToSet" :{"alternative_names" : name}})
            return f[1], f[2]
        elif response == 'n':
            return 'delete',0
        else:
            db.countries.update_one({"alpha-2" : response}, {"$addToSet" :{"alternative_names" : name}})
            return response, db.countries.find_one({"alpha-2" : response})['name']
            
    

In [183]:
import re

collections = ['alcohol', 'average_income', 'child_mortality', 'criminality', 'smokers', 'social_protection' 
               'divorce', 'female_school_achievement', 'fertility', 'first_maternity', 
               'life_expectancy', 'purchasing_power', 'teen_mothers', 'unemployement']

def clean_collection(collection):
    documents = db[collection].find({})
    for document in documents:
        print(document['country'])
        field = list(document.keys())
        for key in ['_id', 'country', 'alpha-2']:
            if key in field: field.remove(key)
        assert(len(field) == 1)
        field = field[0]
        initial_country = document['country']
        dirty_value = ''.join(document[field].split())
        dirty_value = re.findall(r"[-+]?\d*\.\d+|\d+", dirty_value)
        print(dirty_value)
        iso, country = find_country_code(initial_country)
        if len(dirty_value) == 0 or iso == "delete":
            db[collection].delete_one({"country": initial_country})
            print("Missing value deleted")
        else:
            value = float(dirty_value[0])
            db[collection].update_one({"country":initial_country},{"$set" :{"country":country,"alpha-2":iso,field:value}})
    


In [184]:
clean_collection("average_income")

Norvège
['6865']
Suisse
['6770']
Luxembourg
['5965']
Danemark
['4751']
États-Unis
['4734']
Islande
['4732']
Suède
['4540']
Australie
['4519']
Irlande
['4334']
Singapour
['4323']
Pays-Bas
['3884']
Autriche
['3822']
Finlande
['3754']
Allemagne
['3661']
Canada
['3640']
Royaume-Uni
['3530']
Belgique
['3488']
Émirats Arabes Unis
['3373']
Nouvelle-Zélande
['3228']
France
['3226']
Japon
['3160']
Koweït
['2907']
Brunei
['2738']
Italie
['2643']
Corée du Sud
['2300']
Espagne
['2298']
Bahamas
['2207']
Chypre
['2026']
Malte
['2015']
Arabie Saoudite
['1810']
Slovénie
['1801']
Portugal
['1656']
Grèce
['1587']
Estonie
['1480']
Tchéquie
['1460']
Slovaquie
['1400']
Trinité-et-Tobago
['1353']
Saint-Christophe-et-Niévès
['1307']
Seychelles
['1284']
Uruguay
['1269']
Barbade
['1267']
Lituanie
['1230']
Lettonie
['1214']
Antigua-et-Barbuda
['1130']
Chili
['1128']
Pologne
['1056']
Hongrie
['1047']
Îles Palaos
['1027']
SV El Salvador =? Îles Palaos
n
Missing value deleted
Panama
['1011']
Croatie
['1010']



AttributeError: 'NoneType' object has no attribute 'split'